In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model_name = "bigscience/bloomz-560m"
# model_name = "bigscience/bloom-1b1"
NUM_VIRTUAL_TOKENS = 4
NUM_EPOCHS = 6

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

C:\Users\liuhz\.conda\envs\lhz\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\liuhz\.cache\huggingface\hub\models--bigscience--bloomz-560m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [5]:
# this function returns the outputs from the model received, and inputs.
def get_outputs(model,inputs,max_new_tokens=100):
    outputs = model.generate(
        input_ids = inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        # temperature = 2.0,
        # top_P = 0.95,
        # do_sample=True,
        repetition_penalty=1.5,  # Avoid repetition
        early_stopping=True,  # The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

微调之前

In [7]:
input_prompt = tokenizer("I want you to act as a motivational coach. ",return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,input_prompt,max_new_tokens=50)
print(tokenizer.batch_decode(foundational_outputs_prompt,skip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


["I want you to act as a motivational coach.  Don't be afraid of being challenged."]


In [8]:
input_sentences = tokenizer("There are two nice things that should matter to you:", return_tensors="pt")
foundational_outputs_sentence = get_outputs(foundational_model, input_sentences, max_new_tokens=50)
print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


['There are two nice things that should matter to you: the price and quality of your product.']


准备数据集
https://huggingface.co/datasets/fka/awesome-chatgpt-prompts
https://huggingface.co/datasets/Abirate/english_quotes

In [9]:
import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [10]:
from datasets import load_dataset

dataset_prompt = "fka/awesome-chatgpt-prompts"

# Create the Dataset to create prompts
data_prompt = load_dataset(dataset_prompt)
data_prompt = data_prompt.map(lambda samples: tokenizer(samples["prompt"]),batched=True)
train_sample_prompt = data_prompt["train"].select(range(50))

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

C:\Users\liuhz\.conda\envs\lhz\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\liuhz\.cache\huggingface\hub\datasets--fka--awesome-chatgpt-prompts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


prompts.csv:   0%|          | 0.00/104k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [11]:
display(train_sample_prompt)

Dataset({
    features: ['act', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 50
})

In [12]:
print(train_sample_prompt[:1])

{'act': ['An Ethereum Developer'], 'prompt': ['Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.'], 'input_ids': [[186402, 1152, 1306, 660, 72560, 28857, 167625, 84544, 20165, 376, 1002, 26168, 267, 30479, 17477, 613, 267, 120755, 238776, 17, 1387, 47881, 632, 427, 14565, 29866, 664, 368, 120755, 15, 16997, 4054, 136044, 375, 4859, 12, 427, 39839, 15, 9697, 1242, 375, 13614, 12, 3804, 427, 368, 2298, 5268, 109891, 368, 17477, 15, 530, 427, 11210, 4143, 7112, 11866, 3

In [15]:
dataset_sentences = load_dataset("Abirate/english_quotes")
data_sentences = dataset_sentences.map(lambda samples:tokenizer(samples["quote"]),batched=True)
train_sample_sentences = data_sentences["train"].select(range(25))
train_sample_sentences = train_sample_sentences.remove_columns(["author","tags"])

In [16]:
display(train_sample_sentences)

Dataset({
    features: ['quote', 'input_ids', 'attention_mask'],
    num_rows: 25
})

PEFT配置

In [17]:
from peft import get_peft_model,PromptTuningConfig,TaskType,PromptTuningInit

generation_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM, # This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.RANDOM, # The added virtual tokens are initialized with random numbers
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, # Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name
)

创建2个提示微调模型

In [18]:
peft_model_prompt = get_peft_model(foundational_model,generation_config)
print(peft_model_prompt.print_trainable_parameters())

trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007
None


In [19]:
peft_model_sentences = get_peft_model(foundational_model,generation_config)
print(peft_model_sentences.print_trainable_parameters())

trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007
None


In [20]:
from transformers import TrainingArguments

def create_training_arguments(path,learning_rate=0.0035,epochs=6):
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        use_cpu=True,  # This is necessary for CPU clusters
        auto_find_batch_size=True,  # Find a suitable batch size that will fit into memory automatically
        learning_rate=learning_rate,
        num_train_epochs=epochs
    )
    return training_args

In [21]:
working_dir = "./"

output_directory_prompt = os.path.join(working_dir,"peft_outputs_prompt")
output_directory_sentences = os.path.join(working_dir,"peft_outputs_sentences")

os.makedirs(output_directory_prompt,exist_ok=True)
os.makedirs(output_directory_sentences,exist_ok=True)

In [22]:
training_args_prompt = create_training_arguments(output_directory_prompt,0.003,NUM_EPOCHS)
training_args_sentences = create_training_arguments(output_directory_sentences,0.003,NUM_EPOCHS)

训练

In [24]:
from transformers import Trainer,DataCollatorForLanguageModeling

def create_trainer(model,training_args,train_dataset):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer,mlm=False
        ), # mlm=False indicates not to use masked language modeling
    )
    return trainer

In [25]:
# 训练第一个模型
trainer_prompt = create_trainer(peft_model_prompt,training_args_prompt,train_sample_prompt)
trainer_prompt.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


TrainOutput(global_step=42, training_loss=3.4480707077752974, metrics={'train_runtime': 355.1765, 'train_samples_per_second': 0.845, 'train_steps_per_second': 0.118, 'total_flos': 64653968375808.0, 'train_loss': 3.4480707077752974, 'epoch': 6.0})

In [26]:
# 训练第二个模型
trainer_sentences = create_trainer(peft_model_sentences,training_args_sentences,train_sample_sentences)
trainer_sentences.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


TrainOutput(global_step=24, training_loss=4.172862688700358, metrics={'train_runtime': 101.6273, 'train_samples_per_second': 1.476, 'train_steps_per_second': 0.236, 'total_flos': 18779108253696.0, 'train_loss': 4.172862688700358, 'epoch': 6.0})

In [27]:
trainer_prompt.model.save_pretrained(output_directory_prompt)
trainer_sentences.model.save_pretrained(output_directory_sentences)

推理

In [30]:
from peft import PeftModel

loaded_model_prompt = PeftModel.from_pretrained(
    foundational_model,
    output_directory_prompt,
    #device_map="auto",
    is_trainable=False,
)

In [31]:
loaded_model_prompt_outputs = get_outputs(loaded_model_prompt,input_prompt)
print(tokenizer.batch_decode(loaded_model_prompt_outputs,skuip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


["I want you to act as a motivational coach.  Don't be afraid of being rude or mean.</s>"]


In [32]:
loaded_model_prompt.load_adapter(output_directory_sentences,adapter_name="quotes")
loaded_model_prompt.set_adapter("quotes")

In [33]:
loaded_model_sentences_outputs = get_outputs(loaded_model_prompt,input_sentences)
print(tokenizer.batch_decode(loaded_model_sentences_outputs,skip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


['There are two nice things that should matter to you: the weather and your health.']
